In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/datasets'
%ls

/content/drive/MyDrive/Colab Notebooks/datasets
mega_data.csv  megamillions.csv  powerball.csv  power_data.csv


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [4]:

df = pd.read_csv("powerball.csv")
df = df.iloc[:, 4:]
df = df.iloc[:, :-1]
number_of_features = df.shape[1]
df1 = df.copy()
print(list(df.columns))


['37', '52', '22', '36', '17', '24']


In [5]:
print(df.shape)
print(list(df1.columns))

(1482, 6)
['37', '52', '22', '36', '17', '24']


In [6]:
window_length = 7
number_of_features = df.shape[1]

In [7]:
train = df.copy()
train.head((window_length+1))

,37,52,22,36,17,24
0,22,54,52,14,59,4
1,29,8,37,38,5,34
2,14,10,40,51,30,1
3,36,7,26,8,19,15
4,13,54,41,37,27,32
5,50,17,57,35,4,12
6,47,18,58,53,51,30
7,14,49,9,7,45,23


In [8]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [9]:
train_samples[0]


array([[22., 54., 52., 14., 59.,  4.],
       [29.,  8., 37., 38.,  5., 34.],
       [14., 10., 40., 51., 30.,  1.],
       [36.,  7., 26.,  8., 19., 15.],
       [13., 54., 41., 37., 27., 32.],
       [50., 17., 57., 35.,  4., 12.],
       [47., 18., 58., 53., 51., 30.]])

In [10]:
train_labels[0]

array([14., 49.,  9.,  7., 45., 23.])

In [11]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [12]:
scaled_train_samples.head(window_length+1)

,0,1,2,3,4,5
0,-0.594887,1.106990,0.977920,-1.038647,1.326898,-1.271238
1,-0.223678,-1.271381,0.188696,0.230378,-1.491580,1.986355
2,-1.019126,-1.167973,0.346541,0.917767,-0.186729,-1.596997
3,0.147532,-1.323085,-0.390067,-1.355903,-0.760864,-0.076787
4,-1.072156,1.106990,0.399156,0.177502,-0.343311,1.769182
5,0.889951,-0.806048,1.240994,0.071750,-1.543774,-0.402547
6,0.730861,-0.754344,1.293609,1.023519,0.909345,1.552010
7,-1.019126,0.848471,-1.284520,-1.408779,0.596181,0.791904


In [13]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [15]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
pd.options.mode.chained_assignment = None

In [18]:
model.fit(x=x_train, y=y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/400
15/15 - 22s - loss: 0.9948 - accuracy: 0.2312 - 22s/epoch - 1s/step
Epoch 2/400
15/15 - 0s - loss: 0.9832 - accuracy: 0.2488 - 215ms/epoch - 14ms/step
Epoch 3/400
15/15 - 0s - loss: 0.9808 - accuracy: 0.2529 - 205ms/epoch - 14ms/step
Epoch 4/400
15/15 - 0s - loss: 0.9797 - accuracy: 0.2495 - 195ms/epoch - 13ms/step
Epoch 5/400
15/15 - 0s - loss: 0.9790 - accuracy: 0.2441 - 200ms/epoch - 13ms/step
Epoch 6/400
15/15 - 0s - loss: 0.9786 - accuracy: 0.2508 - 208ms/epoch - 14ms/step
Epoch 7/400
15/15 - 0s - loss: 0.9781 - accuracy: 0.2522 - 198ms/epoch - 13ms/step
Epoch 8/400
15/15 - 0s - loss: 0.9780 - accuracy: 0.2502 - 198ms/epoch - 13ms/step
Epoch 9/400
15/15 - 0s - loss: 0.9779 - accuracy: 0.2508 - 198ms/epoch - 13ms/step
Epoch 10/400
15/15 - 0s - loss: 0.9762 - accuracy: 0.2549 - 204ms/epoch - 14ms/step
Epoch 11/400
15/15 - 0s - loss: 0.9763 - accuracy: 0.2488 - 265ms/epoch - 18ms/step
Epoch 12/400
15/15 - 0s - loss: 0.9767 - accuracy: 0.2542 - 284ms/epoch - 19ms/step
Epoc

In [19]:
next_Date = '2/06/2023'
print('-' * 40)
print('Predict the Future Drawing on June 02, 2023')
next = df.copy()
next = next.tail((window_length))
next = np.array(next)
x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
print('Drawing  Date', next_Date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_next_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]+1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]-1)
print('-' * 40)

----------------------------------------
Predict the Future Drawing on June 02, 2023
1/1 [==============================] - 3s 3s/step
Drawing  Date 2/06/2023
Prediction without rounding up or down:	 [45 59 38 21 61 11]
Prediction with rounding up           :	 [46 60 39 22 62 12]
Prediction with rounding down         :	 [44 58 37 20 60 10]
----------------------------------------
